In [1]:
import sys
from pathlib import Path
import qlib
import pandas as pd
from qlib.config import REG_CN
from qlib.contrib.model.gbdt import LGBModel
from qlib.contrib.data.handler import Alpha360
from qlib.utils import init_instance_by_config
from qlib.contrib.strategy import TopkDropoutStrategy
from qlib.contrib.report import analysis_model, analysis_position
# from qlib.contrib.evaluate import (
#     backtest as normal_backtest,
#     risk_analysis,
# )
from qlib.utils import exists_qlib_data, init_instance_by_config, flatten_dict
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict
from qlib.data.dataset.loader import QlibDataLoader
from qlib.contrib.data.handler import Alpha158   #Alpha158内置指标体系
from qlib.data.dataset.loader import QlibDataLoader
import qlib
from qlib.contrib.data.handler import Alpha158   #Alpha158内置指标体系
provider_uri = "./qlib_data/cn_data"  # 原始行情数据存放目录
qlib.init(provider_uri=provider_uri, region=REG_CN)  # 初始化
market = "csi100"
benchmark = "SH000300"

 #数据处理器参数配置
data_handler_config = {
    "start_time": "2020-01-01",
    "end_time": "2020-02-21",
    "fit_start_time": "2020-01-01",  # 模型跑数据的开始时间
    "fit_end_time": "2020-01-31",
    "instruments": market,
    "infer_processors" : [
                                    {'class': 'FilterCol',##过滤器
                                     'kwargs': {'fields_group': 'feature', 'col_list': ["KMID","KLOW","OPEN0"]
                                     }},
                                    {'class': 'RobustZScoreNorm', # RobustZScoreNorm和Fillna，用于进行标准化和填充缺失值。
                                     'kwargs': {'fields_group': 'feature', 'clip_outlier': True}},
                                    {'class': 'Fillna', 'kwargs': {'fields_group': 'feature'}}],
    "learn_processors": [{'class': 'DropnaLabel'}, #DropnaLabel删除标注中含有缺失值的样本。

                                    # 对预测的目标进行截面排序处理  DropnaLabel 和 CSRankNorm 预处理器，用于对标签进行处理。
                                    {'class': 'CSRankNorm', 'kwargs': {'fields_group': 'label'}}],

                                    # 预测的目标
                                    'label': ["Ref($close, -2) / Ref($close, -1) - 1"] ,#下一日收益率, Ref($close, -1)表示下一日收盘价
                                    #'label': ['Ref($close, -1)'],
}
    


# 任务参数配置
task = {
    "model": {  # 模型参数配置
        # 模型类
        "class": "TransformerModel",
        # 模型类所在模块
        "module_path": "qlib.contrib.model.pytorch_transformer_ts",
        "kwargs": {  # 模型超参数配置
            "seed": 0,
            "n_jobs": 20,
        }, 
    },
    "dataset": {  # 　因子库数据集参数配置
        # 数据集类，是Dataset with Data(H)andler的缩写，即带数据处理器的数据集
        "class": "TSDatasetH",
        # 数据集类所在模块
        "module_path": "qlib.data.dataset",
        "kwargs": {  # 数据集参数配置
            "handler": {  # 数据集使用的数据处理器配置
                #"class": "Alpha158",  # 数据处理器类，继承自DataHandlerLP
                "module_path": "qlib.contrib.data.handler",  # 数据处理器类所在模块
                "class": "Alpha158",
                "kwargs": data_handler_config,  # 数据处理器参数配置
            },
             "segments": {  # 数据集划分标准
                "train": ("2020-01-01", "2020-01-31"),  # 此时段的数据为训练集
                "valid": ("2020-01-31", "2020-02-20"),  # 此时段的数据为验证集
                "test": ("2020-02-20", "2020-02-21"),  # 此时段的数据为测试集
            },
        },
    },

}


# 实例化模型对象
model = init_instance_by_config(task["model"])

# 实例化因子库数据集，从基础行情数据计算出的包含所有特征（因子）和标签值的数据集。
dataset = init_instance_by_config(task["dataset"])  # DatasetH


ModuleNotFoundError. CatBoostModel are skipped. (optional: maybe installing CatBoostModel can fix it.)
ModuleNotFoundError. XGBModel is skipped(optional: maybe installing xgboost can fix it).


/home/shared/anaconda3/envs/seg/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[29130:MainThread](2023-04-11 10:18:56,494) INFO - qlib.Initialization - [config.py:413] - default_conf: client.
[29130:MainThread](2023-04-11 10:18:56,499) INFO - qlib.workflow - [expm.py:31] - experiment manager uri is at file:/home/shared/qlib-main/mlruns
[29130:MainThread](2023-04-11 10:18:56,500) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[29130:MainThread](2023-04-11 10:18:56,502) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/home/shared/qlib-main/qlib_data/cn_data')}
[29130:MainThread](2023-04-11 10:18:56,683) INFO - qlib.TransformerModel - [pytorch_transformer_ts.py:62] - Naive Transformer:
batch_size : 256
device :

In [2]:
# start exp to train model
with R.start(experiment_name="train_model"):
    R.log_params(**flatten_dict(task))
    model.fit(dataset)
    R.save_objects(trained_model=model)
    rid = R.get_recorder().id

[29130:MainThread](2023-04-11 10:19:06,131) INFO - qlib.workflow - [expm.py:316] - <mlflow.tracking.client.MlflowClient object at 0x7f27fc339490>
[29130:MainThread](2023-04-11 10:19:06,139) INFO - qlib.workflow - [exp.py:260] - Experiment 1 starts running ...
[29130:MainThread](2023-04-11 10:19:06,599) INFO - qlib.workflow - [recorder.py:339] - Recorder d3bf223e91c04c9c922df45febcc021c starts running under Experiment 1 ...
Not a git repository
To compare two paths outside a working tree:
usage: git diff [--no-index] <path> <path>
[29130:MainThread](2023-04-11 10:19:06,704) INFO - qlib.workflow - [recorder.py:372] - Fail to log the uncommitted code of $CWD when run `git diff`
fatal: 不是一个 git 仓库（或者直至挂载点 / 的任何父目录）
停止在文件系统边界（未设置 GIT_DISCOVERY_ACROSS_FILESYSTEM）。
[29130:MainThread](2023-04-11 10:19:06,808) INFO - qlib.workflow - [recorder.py:372] - Fail to log the uncommitted code of $CWD when run `git status`
Not a git repository
To compare two paths outside a working tree:
[29130:MainThre

In [3]:
###################################
# prediction, backtest & analysis
###################################
port_analysis_config = {
    "executor": {
        "class": "SimulatorExecutor",
        "module_path": "qlib.backtest.executor",
        "kwargs": {
            "time_per_step": "day",
            "generate_portfolio_metrics": True,
        },
    },
    "strategy": {
        "class": "TopkDropoutStrategy",
        "module_path": "qlib.contrib.strategy",
        "kwargs": {
            "model": model,
            "dataset": dataset,
            "topk": 50,
            "n_drop": 5,
        },
    },
    "backtest": {
        "start_time": "2017-01-01",
        "end_time": "2020-08-01",
        "account": 100000000,
        "benchmark": benchmark,
        "exchange_kwargs": {
            "freq": "day",
            "limit_threshold": 0.095,
            "deal_price": "close",
            "open_cost": 0.0005,
            "close_cost": 0.0015,
            "min_cost": 5,
        },
    },
}

# backtest and analysis
with R.start(experiment_name="backtest_analysis"):
    recorder = R.get_recorder(recorder_id=rid, experiment_name="train_model")
    model = recorder.load_object("trained_model")

    # prediction
    recorder = R.get_recorder()
    ba_rid = recorder.id
    sr = SignalRecord(model, dataset, recorder)
    sr.generate()

    # backtest & analysis
    par = PortAnaRecord(recorder, port_analysis_config, "day")
    par.generate()

[29130:MainThread](2023-04-11 10:19:22,165) INFO - qlib.workflow - [expm.py:316] - <mlflow.tracking.client.MlflowClient object at 0x7f27d9861ca0>
[29130:MainThread](2023-04-11 10:19:22,172) INFO - qlib.workflow - [exp.py:260] - Experiment 2 starts running ...
[29130:MainThread](2023-04-11 10:19:22,401) INFO - qlib.workflow - [recorder.py:339] - Recorder 0cd12e1506d144378fe1a514fd1128d0 starts running under Experiment 2 ...
Not a git repository
To compare two paths outside a working tree:
usage: git diff [--no-index] <path> <path>
[29130:MainThread](2023-04-11 10:19:22,539) INFO - qlib.workflow - [recorder.py:372] - Fail to log the uncommitted code of $CWD when run `git diff`
fatal: 不是一个 git 仓库（或者直至挂载点 / 的任何父目录）
停止在文件系统边界（未设置 GIT_DISCOVERY_ACROSS_FILESYSTEM）。
[29130:MainThread](2023-04-11 10:19:22,676) INFO - qlib.workflow - [recorder.py:372] - Fail to log the uncommitted code of $CWD when run `git status`
Not a git repository
To compare two paths outside a working tree:
usage: git diff

'The following are prediction results of the TransformerModel model.'
                          score
datetime   instrument          
2020-02-20 SH600000   -0.021657
           SH600009    0.020816
           SH600010   -0.012387
           SH600011    0.006728
           SH600015    0.018288


[29130:MainThread](2023-04-11 10:19:31,317) WARNING - qlib.online operator - [exchange.py:216] - factor.day.bin file not exists or factor contains `nan`. Order using adjusted_price.
[29130:MainThread](2023-04-11 10:19:31,319) WARNING - qlib.online operator - [exchange.py:218] - trade unit 100 is not supported in adjusted_price mode.
[29130:MainThread](2023-04-11 10:19:36,782) WARNING - qlib.data - [data.py:662] - load calendar error: freq=day, future=True; return current calendar!
[29130:MainThread](2023-04-11 10:19:36,786) WARNING - qlib.data - [data.py:665] - You can get future calendar by referring to the following document: https://github.com/microsoft/qlib/blob/main/scripts/data_collector/contrib/README.md
[29130:MainThread](2023-04-11 10:19:36,813) WARNING - qlib.BaseExecutor - [executor.py:121] - `common_infra` is not set for <qlib.backtest.executor.SimulatorExecutor object at 0x7f27d8bb3d30>
backtest loop: 100%|██████████| 871/871 [00:01<00:00, 453.88it/s]
[29130:MainThread](20

'The following are analysis results of benchmark return(1day).'
                       risk
mean               0.000477
std                0.012295
annualized_return  0.113561
information_ratio  0.598699
max_drawdown      -0.370479
'The following are analysis results of the excess return without cost(1day).'
                       risk
mean              -0.000372
std                0.011079
annualized_return -0.088554
information_ratio -0.518115
max_drawdown      -0.414192
'The following are analysis results of the excess return with cost(1day).'
                       risk
mean              -0.000373
std                0.011079
annualized_return -0.088741
information_ratio -0.519198
max_drawdown      -0.414876
'The following are analysis results of indicators(1day).'
     value
ffr    1.0
pa     0.0
pos    0.0


In [4]:

# 从实验记录器加载保存在pkl文件中的预测结果数据
pred_df = recorder.load_object("pred.pkl")

# 从实验记录器加载保存在pkl文件中的标签数据
label_df = recorder.load_object("label.pkl")
label_df.columns = ['label']

# 构造预测值和标签值并列的df
pred_label = pd.concat([pred_df, label_df], axis=1, sort=True).reindex(label_df.index)

print(pred_label)


                          score     label
datetime   instrument                    
2020-02-20 SH600000   -0.021657 -0.012389
           SH600009    0.020816 -0.038657
           SH600010   -0.012387 -0.008197
           SH600011    0.006728 -0.014000
           SH600015    0.018288 -0.005563
...                         ...       ...
2020-02-21 SZ002594   -0.076613  0.070242
           SZ002736   -0.038421 -0.002357
           SZ300015    0.000201 -0.007494
           SZ300059   -0.069095  0.069132
           SZ300498   -0.047155  0.006787

[200 rows x 2 columns]


In [5]:
import numpy as np
# 计算预测值和标签值的正负号
pred_sign = np.sign(pred_label["score"])
label_sign = np.sign(pred_label["label"])

# 统计正负号相同的数量
same_sign_count = np.sum(pred_sign == label_sign)
same_sign_count

73

In [6]:
# 计算出两列数据正负号相同的比例
corr_pct = (pred_label['score'] * pred_label['label'] > 0).mean()
corr_pct


0.365